In [1]:
import torch 
import torch.nn as nn
import fasttext as ft
import math

In [ ]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
# !gunzip cc.en.300.bin cc.en.300.bin.gz
!gunzip cc.fr.300.bin cc.fr.300.bin.gz

In [ ]:
en = ft.load_model(str('cc.en.300.bin'))
fr = ft.load_model(str('cc.fr.300.bin'))

# New Implementation

In [2]:
d_model = 5

In [111]:
def getWordVectors(sentence):
    sentence = sentence.split(' ')
    vecs = torch.rand((len(sentence),d_model))
    return vecs

In [112]:
def PositionalEncoding(wordVecs):
    for pos in range(wordVecs.shape[0]):
        for i in range(wordVecs[pos].shape[0]):
            if i%2 == 0:
                wordVecs[pos][i] = wordVecs[pos][i] + math.sin(pos/(10000**(2*i/d_model)))
            else:
                wordVecs[pos][i] = wordVecs[pos][i] + math.cos(pos/(10000**(2*i/d_model)))
                
    return wordVecs
                                           

In [113]:
def get_qkv_weights(r,c):
    query_weights = torch.rand((r,c))
    key_weights = torch.rand((r,c))
    value_weights = torch.rand((r,c))
    return query_weights, key_weights, value_weights
    

In [114]:
def qkvs(vectorMatrix, new_dim):
    
    query_weights, key_weights, value_weights = get_qkv_weights(5,new_dim)
    
    return torch.matmul(vectorMatrix, query_weights), torch.matmul(vectorMatrix, key_weights), \
    torch.matmul(vectorMatrix, value_weights) 

In [115]:
def qk_dotproducts(queries, keys):
    dotproduct_matrix = torch.Tensor([])
    for i in queries:
        dotproduct_vector = torch.Tensor([])
        for j in keys:
            dotproduct_vector = torch.cat([dotproduct_vector, torch.dot(i,j).reshape(-1)])
        dotproduct_matrix = torch.cat([dotproduct_matrix, dotproduct_vector.reshape(1,-1)])
     
    return dotproduct_matrix

In [ ]:
'''
wordVecs = getWordVectors('Hi there this is nuts')
pos_encoded = PositionalEncoding(wordVecs)

new_dim = 3
queries, keys, values = qkvs(pos_encoded, new_dim)
qk_dotproductmatrix = qk_dotproducts(queries, keys)


'''

In [116]:
wordVecs = getWordVectors('Hi there this is nuts')
# wordVecs

In [117]:
pos_encoded = PositionalEncoding(wordVecs)
# pos_encoded

In [118]:
new_dim = 3
queries, keys, values = qkvs(pos_encoded, new_dim)

In [119]:
qk_dotproductmatrix = qk_dotproducts(queries, keys)

In [120]:
queries

tensor([[1.9581, 1.4281, 2.6495],
        [2.5119, 1.1949, 2.5609],
        [2.5344, 1.4749, 2.8377],
        [2.2804, 1.1585, 2.7134],
        [1.1326, 0.5787, 1.7512]])

In [121]:
keys

tensor([[4.1901, 1.6701, 2.2903],
        [3.4275, 2.0660, 2.3883],
        [4.0120, 1.9227, 2.8416],
        [3.7320, 2.1417, 2.4647],
        [2.7755, 1.2663, 1.6849]])

In [122]:
qk_dotproductmatrix

tensor([[16.6581, 15.9901, 18.1307, 16.8966, 11.7073],
        [18.3860, 17.1947, 19.6522, 18.2454, 12.7997],
        [19.5820, 18.5115, 21.0675, 19.6114, 13.6832],
        [17.7044, 16.6902, 19.0867, 17.6793, 12.3680],
        [ 9.7232,  9.2603, 10.6330,  9.7826,  6.8270]])